# 금융 문서 요약 (Financial Document Summarization)

LangChain의 **LCEL**을 사용하여 금융 문서를 분석하고 요약하는 시스템을 구축합니다.

## 학습 목표

- LCEL (LangChain Expression Language) 이해
    - **목적**: 결정적이고 예측 가능한 체인 구성
    - **특징**: 
        - 파이프(`|`) 연산자로 컴포넌트 연결
        - 명확한 데이터 흐름
        - 빠른 실행 속도
        - 디버깅 용이

- LCEL 방식의 요약 체인 구성

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# 환경 변수 로드
load_dotenv()

In [ ]:
# 예시 문서 로드
loader = PyPDFLoader("data/금융안정보고서(2025년 6월).pdf")
docs = loader.load()

print(f"페이지 수: {len(docs)}")

In [ ]:
# 32  페이지 내용
print(docs[31].page_content)

In [ ]:
# 32 페이지 메타데이터
print(docs[31].metadata)


## 1. 기본 요약 체인 (Simple Chain)

- **목적**: 간단하고 빠른 요약

- LCEL 기본 구조

    - 파이프 연산자(`|`): 컴포넌트 연결
    - 컴포넌트: LLM, Tool, Prompt, OutputParser 등

        ```python
        # 기본 패턴
        chain = prompt | model | output_parser

        # 요약 체인 예시
        basic_summary_chain = (
            basic_summary_prompt      # 1. 프롬프트 템플릿
            | llm                     # 2. 언어 모델
            | StrOutputParser()       # 3. 출력 파서
        )
        ```

In [ ]:
### 기본 요약 체인 구성

# 프롬프트 템플릿
basic_summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 금융 문서 요약 전문가입니다."),
    ("human", "다음 텍스트를 간결하게 요약하세요:\n\n{text}")
])

# LLM 초기화
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

# 출력 파서
ouput_parser = StrOutputParser()

# 기본 요약 체인
basic_summary_chain = basic_summary_prompt | llm | ouput_parser


In [ ]:
# 예시 문서 요약 (32 페이지)

summary = basic_summary_chain.invoke({"text": docs[31].page_content})
print(summary)


**특징**:
- 단일 입력 → 단일 출력
- 가장 빠른 실행 속도
- 명확한 목적

**사용 시점**:
- 빠른 개요가 필요할 때
- 단순한 요약이 충분할 때

## 2. 구조화된 요약 체인 (Structured Chain)

- **목적**: 일관된 형식의 상세 분석


In [ ]:
structured_summary_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 금융 분석가입니다. 다음 형식으로 요약하세요:
    
    [핵심 요약]
    [재무 지표]
    [주요 동인]
    [리스크 요인]
    """),
    ("human", "{text}")
])

structured_summary_chain = structured_summary_prompt | llm | StrOutputParser()

In [ ]:
# 예시 문서 요약 (32 페이지)

summary = structured_summary_chain.invoke({"text": docs[31].page_content})
print(summary)

**특징**:
- 프롬프트에서 구조 정의
- 일관된 출력 형식
- 가독성 높은 결과

**사용 시점**:
- 보고서 작성 시
- 팀 간 일관된 포맷 필요 시

## 3. 병렬 처리 체인 (Parallel Chain)

- **목적**: 여러 관점을 동시에 분석

In [ ]:
financial_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 재무 분석가입니다. 아래의 데이터를 분석하여 재무지표를 분석하세요."),
    ("user", "{data}")
])


risk_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 리스크 분석가입니다. 아래의 데이터를 분석하여 핵심 위험요인을 분석하세요."),
    ("user", "{data}")
])


opportunity_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 전략기회 전문가입니다. 아래의 데이터를 분석하여 기회 요인을 찾으세요."),
    ("user", "{data}")
])


parallel_chain = RunnableParallel(
    financial_analysis=financial_prompt | llm | StrOutputParser(),
    risk_analysis=risk_prompt | llm | StrOutputParser(),
    opportunity_analysis=opportunity_prompt | llm | StrOutputParser()
)


summary = parallel_chain.invoke({
    "data": docs[31].page_content
})  

print(summary)

In [ ]:
print(summary['financial_analysis'])

In [ ]:
print(summary['risk_analysis'])

In [ ]:
print(summary['opportunity_analysis'])

## 4. 다단계 처리 체인 (Multi-stage Chain)

- **목적**: 이전 결과를 다음 단계에서 활용


In [ ]:
initial_summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 금융 분석가입니다. 주어진 텍스트를 금융 분석 관점에서 요약해주세요."),
    ("human", "{text}")
])

detailed_analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 금융 보고서 작성 전문가입니다. 분석 결과를 토대로 상세한 분석 보고서를 작성해주세요."),
    ("human", "{summary}")
])

multi_stage_chain = (
    # 1단계: 초기 요약
    {"summary": initial_summary_prompt | llm | StrOutputParser()}
    # 2단계: 요약 결과를 사용한 상세 분석
    | RunnablePassthrough.assign(
        analysis=lambda x: (detailed_analysis_prompt | llm | StrOutputParser()).invoke(x)
    )
)

detailed_report = multi_stage_chain.invoke({"text": docs[31].page_content})
print(detailed_report)  


In [ ]:
print(detailed_report['summary'])

In [ ]:
print(detailed_report['analysis'])

---

## 5. 실습 문제 (Exercises)

### 문제 1: 핵심 요약 (Bullet Points)

주어진 텍스트를 **3-5개의 핵심 요약(Bullet Points)** 로 정리하는 체인을 만드세요.

**요구사항**:
1. `ChatPromptTemplate`을 사용하여 프롬프트를 작성하세요.
2. `ChatOpenAI` 모델을 사용하세요.
3. `StrOutputParser`를 사용하여 문자열로 출력하세요.
4. `docs[31].page_content`를 입력으로 사용하여 결과를 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요
ex1_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 요약 전문가입니다. 다음 텍스트를 3-5개의 핵심 요약(Bullet Points)으로 정리하세요."),
    ("human", "{text}")
])

ex1_chain = ex1_prompt | llm | StrOutputParser()

print(ex1_chain.invoke({"text": docs[31].page_content}))


<details>
<summary>💡 예시 정답</summary>

```python
ex1_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 요약 전문가입니다. 다음 텍스트를 3-5개의 핵심 요약(Bullet Points)으로 정리하세요."),
    ("human", "{text}")
])

ex1_chain = ex1_prompt | llm | StrOutputParser()

print(ex1_chain.invoke({"text": docs[31].page_content}))
```
</details>

### 문제 2: 긍정/부정 신호 분석 (Parallel Chain)

주어진 텍스트에서 **긍정적 신호(Positive Signals)**와 **부정적 신호(Negative Signals)**를 동시에 분석하는 병렬 체인을 만드세요.

**요구사항**:
1. `positive_prompt`와 `negative_prompt`를 각각 작성하세요.
2. `RunnableParallel`을 사용하여 두 분석을 동시에 실행하세요.
3. 결과는 딕셔너리 형태로 출력되어야 합니다 (`positive`, `negative` 키 사용).

In [ ]:
# 여기에 코드를 작성하세요


<details>
<summary>💡 예시 정답</summary>

```python
# 정답 예시
positive_prompt = ChatPromptTemplate.from_messages([
    ("system", "텍스트에서 긍정적인 시장 신호나 지표만 추출하여 요약하세요."),
    ("human", "{text}")
])

negative_prompt = ChatPromptTemplate.from_messages([
    ("system", "텍스트에서 부정적인 시장 신호나 위험 요인만 추출하여 요약하세요."),
    ("human", "{text}")
])

ex2_chain = RunnableParallel(
    positive=positive_prompt | llm | StrOutputParser(),
    negative=negative_prompt | llm | StrOutputParser()
)

result = ex2_chain.invoke({"text": docs[31].page_content})
print("=== 긍정적 신호 ===\n", result['positive'])
print("\n=== 부정적 신호 ===\n", result['negative'])
```
</details>

### 문제 3: 투자 제안 메일 작성 (Multi-stage Chain)

문서를 요약한 후, 그 요약 내용을 바탕으로 고객에게 보낼 **투자 제안 이메일**을 작성하는 체인을 만드세요.

**요구사항**:
1. **1단계**: 텍스트를 요약합니다 (기존 `basic_summary_chain` 활용 가능).
2. **2단계**: 요약된 내용을 바탕으로 "투자 제안 이메일"을 작성합니다.
3. `RunnablePassthrough` 또는 딕셔너리 구조를 사용하여 단계를 연결하세요.

In [ ]:
# 여기에 코드를 작성하세요


<details>
<summary>💡 예시 정답</summary>

```python
email_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 투자 자문가입니다. 다음 요약 내용을 바탕으로 고객에게 투자 전략을 제안하는 정중한 이메일을 작성하세요."),
    ("human", "요약 내용:\n{summary}")
])

ex3_chain = (
    {"summary": basic_summary_chain}
    | email_prompt
    | llm
    | StrOutputParser()
)

print(ex3_chain.invoke({"text": docs[31].page_content}))
```
</details>